In [18]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
from torchinfo import summary

class classifier_LSTM_per_channel(nn.Module):
    def __init__(self, channels_num):
        super(classifier_LSTM_per_channel, self).__init__()
        self.lstm_size0 = 1
        self.channels_num = channels_num
        self.lstm_per_channel = nn.LSTM(1, hidden_size=self.lstm_size0, batch_first=True)
        self.lstm_full1 = nn.LSTM(channels_num, hidden_size=128, batch_first=True)
        self.lstm_full2 = nn.LSTM(channels_num, hidden_size=64, batch_first=True)
        self.linear = nn.Linear(64, out_features=6)
        self.softmax = nn.Softmax()
    def forward(self, X):
        #X: (batch_size, channels_num, sequence_len)
        print(X.size())
        batch_size = len(X)
        lstm_init = (torch.zeros(1, batch_size, self.lstm_size0),
                     torch.zeros(1, batch_size, self.lstm_size0))
        # if x.is_cuda:
        #     lstm_init = (lstm_init[0].cuda(self.GPUindex),
        #                  lstm_init[0].cuda(self.GPUindex))
        lstm_init = (Variable(lstm_init[0]), Variable(lstm_init[1]))
        first_layer_inputs = []
        second_layer_inputs = [] #(32, timesteps)
        for i in range(self.channels_num): 
            x_in = torch.tensor(X[:, i, :]) # x_in: (batch_size, 1, timesteps)
            x_in = torch.squeeze(x_in, 1) #x_in: (batch_size, timesteps)
            x_in = torch.unsqueeze(x_in, -1) #x_in: (batch_size, timesteps, input_size=1)
            first_layer_inputs.append(x_in)
            x_out = self.lstm_per_channel(x_in, lstm_init)[0] # x_out: (batch_size, timesteps, 1)
            second_layer_inputs.append(x_out)

        X = torch.cat(second_layer_inputs, -1) #X: (batch_size, timesteps, 32)
        print("X after concat: ", X.size())
        X = self.lstm_full1(X)
        X = self.lstm_full2(X)
        X = self.linear(X)
        X = self.softmax(X)

        return X

In [19]:
model = classifier_LSTM_per_channel(channels_num=32)
summary(model, input_size=(1, 32, 15360))

torch.Size([1, 32, 15360])


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_22224\4222463860.py:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_in = torch.tensor(X[:, i, :]) # x_in: (batch_size, 1, timesteps)


X after concat:  torch.Size([1, 15360, 32])


RuntimeError: Failed to run torchinfo. See above stack traces for more details. Executed layers up to: [LSTM: 1, LSTM: 1, LSTM: 1, LSTM: 1, LSTM: 1, LSTM: 1, LSTM: 1, LSTM: 1, LSTM: 1, LSTM: 1, LSTM: 1, LSTM: 1, LSTM: 1, LSTM: 1, LSTM: 1, LSTM: 1, LSTM: 1, LSTM: 1, LSTM: 1, LSTM: 1, LSTM: 1, LSTM: 1, LSTM: 1, LSTM: 1, LSTM: 1, LSTM: 1, LSTM: 1, LSTM: 1, LSTM: 1, LSTM: 1, LSTM: 1, LSTM: 1, LSTM: 1]